# Week 04a Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm22prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.
- https://en.wikipedia.org/wiki/Smoothing


Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles

Please add topics you want to learn here: https://padlet.com/ffeenstra1/z9duo25d39dcgezz


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. 
Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [1]:
import re as rg
import numpy as np
import pandas as pd

path = "C:/Users/marsh/OneDrive/Bureaublad/prgr1-main/prgr1-main/KNMI.tsv"
patron = rg.compile("^[\s]{2}[2][7][0]")
raw_data = []
with open(path,"r") as f:
    for line in f:
        line = line.strip("\n")
        if patron.match(line):
            raw_data.append(line)


station =[]
date = []
tmeans = []
tmin = []
tmax = [] 

for data in raw_data:
    data = data.split(",")
    station.append(data[0])
    date.append(data[1])
    tmeans.append(data[2])
    tmin.append(data[3])
    tmax.append(data[4])

df = pd.DataFrame(list(zip(station,date,tmeans,tmin,tmax)),columns=(["station","Date","Tmean","Tmin","Tmax"]))
df["Date"] = pd.to_datetime(df["Date"])

df.head()

            

,station,Date,Tmean,Tmin,Tmax
0,270,2000-01-01,42,-4,79
1,270,2000-01-02,55,33,74
2,270,2000-01-03,74,49,89
3,270,2000-01-04,46,22,75
4,270,2000-01-05,41,14,56


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [2]:
#replace cells with spaces to NaN
#change data formats
#change temperatures to celcius degrees
#remove leap year

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [2]:
#Test your outcome
#write code to check if you have done the above
df = df.replace(r'^\s*$',np.nan,regex=True)
df[["Tmean","Tmax","Tmin"]] = df[["Tmean","Tmax","Tmin"]].astype(float)
df[["Tmean","Tmax","Tmin"]] = df[["Tmean","Tmax","Tmin"]]/10
df["station"] = df["station"].astype(int)
df = df[~((df["Date"].dt.month == 2) & (df["Date"].dt.day == 29))]

df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 6935 entries, 0 to 6939
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  6935 non-null   int32         
 1   Date     6935 non-null   datetime64[ns]
 2   Tmean    6935 non-null   float64       
 3   Tmin     6935 non-null   float64       
 4   Tmax     6935 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int32(1)
memory usage: 298.0 KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [3]:
df_for_2018 = df[(df["Date"] < "2018-1-1")]
df_past_2018 = df[(df["Date"] > "2018-1-1")]

def month_day(df_multipleyears):
    df_multipleyears = pd.DataFrame(df_multipleyears)
    
    df_groupedbymonthday_max = df_multipleyears.groupby([df_multipleyears["Date"].dt.month,(df_multipleyears["Date"].dt.day)])["Tmax"].max()
    df_groupedbymonthday_min = df_multipleyears.groupby([df_multipleyears["Date"].dt.month,(df_multipleyears["Date"].dt.day)])["Tmin"].min()

    return df_groupedbymonthday_max, df_groupedbymonthday_min

max_for_2018, min_for_2018 = month_day(df_for_2018)




In [5]:
#Test your code
def test_reformed(df):
    #
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    month_day(df)

test_reformed(df)

AttributeError: 'Int64Index' object has no attribute 'year'

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [4]:
#your code here
def make_smooth(array):
    array = pd.DataFrame(array)
    array["SMA_mean"] = array["Tmean"].rolling(5).mean()

    return array
smooted_data = make_smooth(df_past_2018)
smooted_data.head(100)

# I used the moving average model to smooth the data because we want to see the difference in temp before and after 2018

,station,Date,Tmean,Tmin,Tmax,SMA_mean
6576,270,2018-01-02,5.6,3.1,7.5,NaN
6577,270,2018-01-03,7.5,5.3,9.2,NaN
6578,270,2018-01-04,7.3,5.8,9.1,NaN
6579,270,2018-01-05,6.0,4.0,7.6,NaN
6580,270,2018-01-06,4.4,1.8,6.1,6.16
...,...,...,...,...,...,...
6671,270,2018-04-07,12.7,6.8,19.3,9.30
6672,270,2018-04-08,11.4,6.4,19.2,9.36
6673,270,2018-04-09,7.9,6.1,11.3,9.10
6674,270,2018-04-10,12.6,7.6,19.0,10.54


---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [5]:
from bokeh.models import Band
from bokeh.plotting import figure ,show
from bokeh.models import DatetimeTickFormatter
from math import pi

fig = figure(title = "average temp past and before 2018",x_axis_type ="datetime", x_axis_label = "Date", y_axis_label = "temperature in degrees celcius")
fig.line(smooted_data["Date"],smooted_data["SMA_mean"],legend_label = " temperture 2018",color = "purple")
fig.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],)
fig.xaxis.major_label_orientation = pi/4
fig.line(smooted_data["Date"],max_for_2018,color= 'red', legend_label = "maximum of past years")
fig.line(smooted_data["Date"],min_for_2018,color= 'blue', legend_label = "minimaal of past years")

fig.legend.title = "Observations"
show(fig)
max_for_2018.head()

Date  Date
1     1       11.1
      2       10.2
      3       10.7
      4        9.8
      5        9.4
Name: Tmax, dtype: float64

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather